In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

다항분포 나이브 베이즈(Multinimial Naive Bayes)  
분류 데이터의 특징이 출현 횟수로 표현되었을 때 데이터의 출현 횟수에 따라 값을 달리하는 데이터에 사용한다.

영화 리뷰와 레이블(긍정/부정)를 활용해 다항분포 나이브 베이즈 분류로 영화 리뷰가 긍정적인지 부정적인지 분류한다.

데이터 획득  
movie_review: 영화 리뷰, type: 리뷰가 긍정적(positive)인지 부정적(negative)인지 나타낸다.

In [2]:
# 학습 데이터
review_list = [
    {'movie_review' : 'this is great great movie. I will watch again', 'type' : 'positive'},
    {'movie_review' : 'I like this movie', 'type' : 'positive'},
    {'movie_review' : 'amazing movie in this year', 'type' : 'positive'},
    {'movie_review' : 'cool my boyfriend also said the movie is cool', 'type' : 'positive'},
    {'movie_review' : 'awesome of the awesome movie ever', 'type' : 'positive'},
    {'movie_review' : 'shame I wasted money and time', 'type' : 'negative'},
    {'movie_review' : 'regret on this move. I will never never what movie from this director', 'type' : 'negative'},
    {'movie_review' : 'I do not like this movie', 'type' : 'negative'},
    {'movie_review' : 'I do not like actors in this movie', 'type' : 'negative'},
    {'movie_review' : 'boring boring sleeping movie', 'type' : 'negative'}
]

# 테스트 데이터
test_feedback_list = [
    {'movie_review': 'great great great movie ever', 'type': 'positive'},
    {'movie_review': 'I like this amazing movie', 'type': 'positive'},
    {'movie_review': 'my boyfriend said great movie ever', 'type': 'positive'},
    {'movie_review': 'cool cool cool', 'type': 'positive'},
    {'movie_review': 'awesome boyfriend said cool movie ever', 'type': 'positive'},
    {'movie_review': 'shame shame shame', 'type': 'negative'},
    {'movie_review': 'awesome director shame movie boring movie', 'type': 'negative'},
    {'movie_review': 'do not like this movie', 'type': 'negative'},
    {'movie_review': 'I do not like this boring movie', 'type': 'negative'},
    {'movie_review': 'aweful terrible boring movie', 'type': 'negative'}
]

In [3]:
# 학습 데이터 준비
df = pd.DataFrame(review_list)
df

,movie_review,type
0,this is great great movie. I will watch again,positive
1,I like this movie,positive
2,amazing movie in this year,positive
3,cool my boyfriend also said the movie is cool,positive
4,awesome of the awesome movie ever,positive
5,shame I wasted money and time,negative
6,regret on this move. I will never never what m...,negative
7,I do not like this movie,negative
8,I do not like actors in this movie,negative
9,boring boring sleeping movie,negative


In [4]:
# 학습 데이터 다듬기
# 사이킷런의 다항분포 나이브 베이즈 분류기는 숫자 데이터만 다루기 때문에 positive를 1, negative를 0으로 
# 치환한 'label' 파생 변수를 추가한다.
df['label'] = df.type.map({'positive': 1, 'negative': 0})
df

,movie_review,type,label
0,this is great great movie. I will watch again,positive,1
1,I like this movie,positive,1
2,amazing movie in this year,positive,1
3,cool my boyfriend also said the movie is cool,positive,1
4,awesome of the awesome movie ever,positive,1
5,shame I wasted money and time,negative,0
6,regret on this move. I will never never what m...,negative,0
7,I do not like this movie,negative,0
8,I do not like actors in this movie,negative,0
9,boring boring sleeping movie,negative,0


학습에 사용할 데이터(피처)와 레이블로 분리한다.

In [5]:
x = df['movie_review'] # 피쳐
x

0        this is great great movie. I will watch again
1                                    I like this movie
2                           amazing movie in this year
3        cool my boyfriend also said the movie is cool
4                    awesome of the awesome movie ever
5                        shame I wasted money and time
6    regret on this move. I will never never what m...
7                             I do not like this movie
8                   I do not like actors in this movie
9                         boring boring sleeping movie
Name: movie_review, dtype: object

In [6]:
y = df['label'] # 레이블
y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

영화 리뷰로 학습을 진행하고 레이블을 사용해서 긍정적, 부정적 여부를 판단한다.

다항분포 나이브 베이즈의 입력 데이터는 고정된(동일한) 크기의 벡터이어야 한다.

In [7]:
# 모든 데이터에 출현한 단어 개수 만큼의 크기를 가지는 벡터를 만들고 고정된 벡터로 표현하기 위해 import 한다.
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
# CountVectorizer 객체는 문자열에 출현한 모든 단어를 오름차순으로 정렬해 단어의 위치로 행렬을 만든다.
# 특정 단어가 출현할 경우 출현한 단어의 개수를 리턴하고 출현하지 않으면 0을 리턴한다.
# binary 옵션의 기본값은 None으로 출현한 단어의 개수를 리턴하고 True로 변경하면 같은 단어가 여러번 출현하더라도
# 무조건 1로 리턴한다.
# cv = CountVectorizer(binary=True) # CountVectorizer 객체를 만든다. 베르누이 나이브 베이즈
cv = CountVectorizer() # 다항분포 나이브 베이즈
# x_train = cv.fit(x) # CountVectorizer 객체를 학습시킨다.
# x_train = cv.transform(x) # CountVectorizer 객체의 학습 결과를 적용한다.
x_train = cv.fit_transform(x) # CountVectorizer 객체를 학습하고 학습 결과를 적용한다.
encoding = x_train.toarray() # CountVectorizer 객체의 학습 결과를 넘파이 배열로 변환한다.
print(type(encoding))
print(encoding)

<class 'numpy.ndarray'>
[[0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  1]
 [0 0 1 0 0 0 0 1 2 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
  0]
 [0 0 0 0 0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0
  0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 2 0 0 1 1 0 0 0 0 2 0 0 0 1 1
  0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0]]


위의 넘파이 배열에서 볼 수 있듯이 영화 리뷰에서 총 37개의 단어가 발견되어 각 영화 리뷰가 37개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.  
binary=True 옵션을 생략해서 다항분포 나이브 베이즈에서 사용하기 위해 영화 리뷰에 중복되서 출현한 단어가 있으면 출현한 횟수로 표현된 것도 알 수 있다.

In [10]:
# inverse_transform() 메소드로 고정된 크기의 벡터에 포함된 단어를 확인할 수 있다.
for string in cv.inverse_transform(x_train):
    print(string)

['this' 'is' 'great' 'movie' 'will' 'watch' 'again']
['this' 'movie' 'like']
['this' 'movie' 'amazing' 'in' 'year']
['is' 'movie' 'cool' 'my' 'boyfriend' 'also' 'said' 'the']
['movie' 'the' 'awesome' 'of' 'ever']
['shame' 'wasted' 'money' 'and' 'time']
['this' 'movie' 'will' 'regret' 'on' 'move' 'never' 'what' 'from'
 'director']
['this' 'movie' 'like' 'do' 'not']
['this' 'movie' 'like' 'in' 'do' 'not' 'actors']
['movie' 'boring' 'sleeping']


In [11]:
# get_feature_names() 메소드로 고정된 벡터의 각 열(피처)이 어떤 단어를 의미하는지 확인할 수 있다.
print(cv.get_feature_names())

['actors', 'again', 'also', 'amazing', 'and', 'awesome', 'boring', 'boyfriend', 'cool', 'director', 'do', 'ever', 'from', 'great', 'in', 'is', 'like', 'money', 'move', 'movie', 'my', 'never', 'not', 'of', 'on', 'regret', 'said', 'shame', 'sleeping', 'the', 'this', 'time', 'wasted', 'watch', 'what', 'will', 'year']


다항분포 나이브 베이즈 모델 학습하기

사이킷런의 다항분포 나이브 베이즈도 베르누이 나이브 베이즈와 같이 라플라스 스무딩을 지원한다.

In [12]:
# 다항분포 나이브 베이즈 모델을 사용하기 위해 import 한다.
from sklearn.naive_bayes import MultinomialNB

In [13]:
# 다항분포 나이브 베이즈 모델을 만들고 학습시킨다.
mnb = MultinomialNB().fit(x_train, y)

테스트 데이터 준비하고 다듬기

In [14]:
# 사이킷런의 다항분포 나이브 베이즈 분류기는 숫자 데이터만 다루기 때문에 positive를 1, negative를 0으로 
# 치환한 'label' 파생 변수를 추가한다.
test_df = pd.DataFrame(test_feedback_list)
test_df['label'] = df.type.map({'positive': 1, 'negative': 0})
test_df

,movie_review,type,label
0,great great great movie ever,positive,1
1,I like this amazing movie,positive,1
2,my boyfriend said great movie ever,positive,1
3,cool cool cool,positive,1
4,awesome boyfriend said cool movie ever,positive,1
5,shame shame shame,negative,0
6,awesome director shame movie boring movie,negative,0
7,do not like this movie,negative,0
8,I do not like this boring movie,negative,0
9,aweful terrible boring movie,negative,0


테스트에 사용할 데이터(피처)와 레이블로 분리한다.

In [15]:
x = test_df['movie_review'] # 피쳐
x

0                 great great great movie ever
1                    I like this amazing movie
2           my boyfriend said great movie ever
3                               cool cool cool
4       awesome boyfriend said cool movie ever
5                            shame shame shame
6    awesome director shame movie boring movie
7                       do not like this movie
8              I do not like this boring movie
9                 aweful terrible boring movie
Name: movie_review, dtype: object

In [16]:
y = test_df['label'] # 레이블
y

0    1
1    1
2    1
3    1
4    1
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64

모델 테스트

In [17]:
# CountVectorizer 객체는 학습 데이터를 다듬을 때 이미 학습을 시켰으므로 테스트 시에는 적용만 시키면 된다.
x_test = cv.transform(x)

In [19]:
# predict() 메소드의 인수로 테스트 데이터의 피쳐를 넘겨서 예측값을 계산한다.
predict = mnb.predict(x_test)
print(predict)
# accuracy_score() 메소드의 인수로 테스트 데이터의 레이블(실제값)과 예측값을 넘겨서 정확도를 계산한다.
accuracy = accuracy_score(y, predict)
print('정확도: {:6.2%}'.format(accuracy))

[1 1 1 1 1 0 0 0 0 0]
정확도: 100.00%


In [20]:
# confusion_matrix() 메소드의 인수로 테스트 데이터의 레이블(실제값)과 예측값을 넘겨서 혼동 행렬을 출력한다.
print(confusion_matrix(y, predict))

[[5 0]
 [0 5]]


In [21]:
# classification_report() 메소드의 인수로 테스트 데이터의 레이블(실제값)과 예측값을 넘겨서 분류 리포트를 출력한다.
print(classification_report(y, predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [22]:
pd.DataFrame({'실제값': y, '예측값': predict})

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
